In [12]:
import pandas as pd
file = r'C:\Users\moham\OneDrive\Desktop\WebScraping\legacy_app\lily_home.csv'
df = pd.read_csv(file)
df = df[df['Description'].fillna('').str.strip() != ""]
df = df.dropna()
df

,Product,Variant,Price,Handle,Variant ID,Availability,SKU,Product URL,Scraped at,Description
129,Bath mat,Almond,720.0,bath-mat,41081319489730,False,lil-Liv-B1-1-1,https://eg.lilly-home.com//products/bath-mat,2026/02/02 01:30:47 PM,650 gm/msquared 100% egyptian cotton
130,Bath mat,Brown,720.0,bath-mat,41081319522498,False,lil-Liv-B1-1-2,https://eg.lilly-home.com//products/bath-mat,2026/02/02 01:30:47 PM,650 gm/msquared 100% egyptian cotton
131,Bath mat,Genzari,720.0,bath-mat,41081319555266,False,lil-Liv-B1-1-3,https://eg.lilly-home.com//products/bath-mat,2026/02/02 01:30:47 PM,650 gm/msquared 100% egyptian cotton
132,Bath mat,Gray,720.0,bath-mat,41081319588034,False,lil-Liv-B1-1-4,https://eg.lilly-home.com//products/bath-mat,2026/02/02 01:30:47 PM,650 gm/msquared 100% egyptian cotton
379,Half face,Black,620.0,half-face,41147521466562,False,lil-NMG-1-2-1-1-2-1-1-1-1-1-1,https://eg.lilly-home.com//products/half-face,2026/02/02 01:32:37 PM,product details : Cement - Handmade
380,Half face,White,620.0,half-face,41147521499330,False,lil-NMG-1-2-1-1-2-1-1-1-1-1-2,https://eg.lilly-home.com//products/half-face,2026/02/02 01:32:37 PM,product details : Cement - Handmade


In [13]:
import pandas as pd

# 1. Clean the 'Description' column BEFORE merging to remove new lines
df['Description'] = df['Description'].str.replace(r'\n|\r', ' ', regex=True)

# 2. Updated aggregation rules to create clean strings instead of Python lists
# We use ', '.join() to make "White, Cream" instead of "['White', 'Cream']"
aggregation_rules = {
    'Variant': lambda x: ', '.join(map(str, x.unique())), 
    'Variant ID': lambda x: ', '.join(map(str, x.unique())),
    'SKU': lambda x: ', '.join(map(str, x.unique())),
    'Handle': 'first',
    'Availability': 'first',
    'Product URL': 'first',
    'Scraped at': 'first'
}

# Group by the "Core" identity
df_merged = df.groupby(['Product', 'Description', 'Price'], as_index=False).agg(aggregation_rules)

# 3. FIX: Create the Hollistic Description correctly
# Use .astype(str) instead of str() to handle it row-by-row
df_merged['Hollistic Description'] = (
    df_merged['Product'] + 
    ' - ' + 
    df_merged['Variant'].astype(str) + 
    ": " + 
    df_merged["Description"]
)

# Reorder columns
columns_order = [
    'Product', 'Variant', 'Price', 'Handle', 'Variant ID', 
    'Availability', 'SKU', 'Product URL', 'Scraped at', 
    'Description', 'Hollistic Description'
]
df_merged = df_merged[columns_order]

# Save to CSV (index=False prevents that extra "0, 1, 2" column at the start)
df_merged.to_csv(r'C:\Users\moham\OneDrive\Desktop\WebScraping\legacy_app\lilly_home_compact_view.csv', index=False)
df_merged

,Product,Variant,Price,Handle,Variant ID,Availability,SKU,Product URL,Scraped at,Description,Hollistic Description
0,Bath mat,"Almond, Brown, Genzari, Gray",720.0,bath-mat,"41081319489730, 41081319522498, 41081319555266...",False,"lil-Liv-B1-1-1, lil-Liv-B1-1-2, lil-Liv-B1-1-3...",https://eg.lilly-home.com//products/bath-mat,2026/02/02 01:30:47 PM,650 gm/msquared 100% egyptian cotton,"Bath mat - Almond, Brown, Genzari, Gray: 650 g..."
1,Half face,"Black, White",620.0,half-face,"41147521466562, 41147521499330",False,"lil-NMG-1-2-1-1-2-1-1-1-1-1-1, lil-NMG-1-2-1-1...",https://eg.lilly-home.com//products/half-face,2026/02/02 01:32:37 PM,product details : Cement - Handmade,"Half face - Black, White: product details : Ce..."


In [14]:
'''
Products Table
'''
df_products = df_merged
df_products = df_products.drop(columns=['Variant', 'Price', 'Variant ID', 'Availability', 'SKU'])
df_products.to_csv(r'C:\Users\moham\OneDrive\Desktop\WebScraping\legacy_app\lilly_home_products.csv', index=False)
df_products

,Product,Handle,Product URL,Scraped at,Description,Hollistic Description
0,Bath mat,bath-mat,https://eg.lilly-home.com//products/bath-mat,2026/02/02 01:30:47 PM,650 gm/msquared 100% egyptian cotton,"Bath mat - Almond, Brown, Genzari, Gray: 650 g..."
1,Half face,half-face,https://eg.lilly-home.com//products/half-face,2026/02/02 01:32:37 PM,product details : Cement - Handmade,"Half face - Black, White: product details : Ce..."


In [15]:
'''
Variants Table
'''

df_variants = df
df_variants = df_variants.drop(columns=['Product', "Product URL", 'Description'])
cols_to_move = ['Variant ID', 'Handle', 'Variant', 'Price', 'SKU', 'Scraped at']

# Get the list of all other columns that aren't in the "move" list
remaining_cols = [c for c in df_variants.columns if c not in cols_to_move]

# Reorder with move-list first, then the rest
df_variants = df[cols_to_move + remaining_cols]
df_variants.to_csv(r'C:\Users\moham\OneDrive\Desktop\WebScraping\legacy_app\lilly_home_variants.csv', index=False)
df_variants

,Variant ID,Handle,Variant,Price,SKU,Scraped at,Availability
129,41081319489730,bath-mat,Almond,720.0,lil-Liv-B1-1-1,2026/02/02 01:30:47 PM,False
130,41081319522498,bath-mat,Brown,720.0,lil-Liv-B1-1-2,2026/02/02 01:30:47 PM,False
131,41081319555266,bath-mat,Genzari,720.0,lil-Liv-B1-1-3,2026/02/02 01:30:47 PM,False
132,41081319588034,bath-mat,Gray,720.0,lil-Liv-B1-1-4,2026/02/02 01:30:47 PM,False
379,41147521466562,half-face,Black,620.0,lil-NMG-1-2-1-1-2-1-1-1-1-1-1,2026/02/02 01:32:37 PM,False
380,41147521499330,half-face,White,620.0,lil-NMG-1-2-1-1-2-1-1-1-1-1-2,2026/02/02 01:32:37 PM,False
